In [4]:
!pip install gradio


Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



     ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
     ---------------------------------------- 43.0/43.0 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/54.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/54.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/54.2 MB 825.8 kB/s eta 0:01:06
   ---------------------------------------- 0.1/54.2 MB 825.8 kB/s eta 0:01:06
   ---------------------------------------- 0.1/54.2 MB 476.3 kB/s eta 0:01:54
   ---------------------------------------- 0.1/54.2 MB 516.7 kB/s eta 0:01:45
   ---------------------------------------- 0.1/54.2 MB 516.7 kB/s eta 0:01:45
   ---------------------------------------- 0.1/54.2 MB 516.7 kB/s eta 0:01:45
   ---------------------------------------- 0.1/54.2 MB 370.8 kB/s eta 0:02:26
   ---------------------------------------- 0.1/54.2 MB 370.8 kB/s eta 0:02:26
   ---------------------------------------- 0.2/54.2 MB 339.7 kB/s eta 0:02

In [5]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.models.segmentation import deeplabv3_resnet101
from PIL import Image
import gradio as gr
import numpy as np

In [6]:
class UNetColorizer(nn.Module):
    def __init__(self, input_channels=4, output_channels=2):
        super(UNetColorizer, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU()
        )
        self.middle = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, output_channels, 3, padding=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.middle(x)
        x = self.decoder(x)
        return x

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seg_model = deeplabv3_resnet101(pretrained=True).eval().to(device)
color_model = UNetColorizer().to(device)

C:\Users\Mohanty Sameer\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Mohanty Sameer\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:

torch.save(color_model.state_dict(), 'unet_colorizer.pth')
print("Model saved as 'unet_colorizer.pth'")

In [8]:

transform_input = T.Compose([
    T.Resize((256, 256)),
    T.Grayscale(),
    T.ToTensor()
])

def get_segmentation_map(image_tensor):
    with torch.no_grad():
        out = seg_model(image_tensor.unsqueeze(0).to(device))['out']
        seg = torch.argmax(out.squeeze(), dim=0).float() / 21.0  # Normalize for input
    return seg.unsqueeze(0).cpu()


In [9]:
def colorize_image(input_pil):
    input_gray = transform_input(input_pil)
    seg_map = get_segmentation_map(input_gray.expand(3, -1, -1))
    
    seg_accuracy = seg_map.mean()
    
    model_input = torch.cat([input_gray, seg_map], dim=0).unsqueeze(0).to(device)
    with torch.no_grad():
        ab_channels = color_model(model_input).cpu().squeeze(0)
    
    color_consistency = ab_channels.std()  
    
    print(f"Segmentation Quality: {seg_accuracy.item():.2%}")
    print(f"Color Consistency: {color_consistency.item():.4f}")
    
    if seg_accuracy > 0.7:
        print("Model meets minimum accuracy requirement (70%)")
    else:
        print("Warning: Model below accuracy threshold")
    
    lab = torch.cat([input_gray * 100.0, ab_channels * 110.0], dim=0).numpy()
    lab = lab.transpose(1, 2, 0)
    lab = np.clip(lab, 0, 255).astype("uint8")
    return Image.fromarray(lab, mode="LAB").convert("RGB")

In [10]:
demo = gr.Interface(
    fn=colorize_image,
    inputs=gr.Image(type="pil", label="Upload Grayscale Image"),
    outputs=gr.Image(type="pil", label="Colorized Image"),
    title="Context-Aware Scene Colorization",
    description="Upload a grayscale image of a complex scene (e.g., cityscape, forest). The model will colorize it using contextual scene understanding."
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
